In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D ,UpSampling2D, Flatten, Input
from tensorflow.keras.optimizers import  Adam
# adam = adam_v2.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from math import log10,sqrt

In [2]:
def autoencoder():
  input_img=Input(shape=(None,None,1),name='image_input')
  #enoder 
  x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
  x = MaxPooling2D((2,2), padding='same', name='pool1')(x)
  x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv2')(x)
  x = MaxPooling2D((2,2), padding='same', name='pool2')(x)
   
  #decoder
  x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3')(x)
  x = UpSampling2D((2,2), name='upsample1')(x)
  x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv4')(x)
  x = UpSampling2D((2,2), name='upsample2')(x)
  x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv5')(x)
   
  #model
  autoencoder = Model(inputs=input_img, outputs=x)
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    
  return autoencoder

In [3]:
model = autoencoder()

2022-04-09 20:31:19.282593: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-09 20:31:19.333673: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff077f29640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-09 20:31:19.333705: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, None, None, 1)]   0         
_________________________________________________________________
Conv1 (Conv2D)               (None, None, None, 64)    640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, None, None, 64)    0         
_________________________________________________________________
Conv2 (Conv2D)               (None, None, None, 64)    36928     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, None, None, 64)    0         
_________________________________________________________________
Conv3 (Conv2D)               (None, None, None, 64)    36928     
_________________________________________________________________
upsample1 (UpSampling2D)     (None, None, None, 64)   

In [5]:
model.layers[5]

In [6]:
get_latentCode_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])


In [10]:
input_random = np.random.rand(1,64,48,1)
layer_output = get_latentCode_output([input_random])[0]
layer_output.shape

(1, 16, 12, 64)